# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [ ]:
%pip install -q langchain-tavily

In [ ]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

In [ ]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [ ]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

In [ ]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

In [ ]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose=True)



In [32]:
agent_executor.invoke({'input': '나는 유태영이야'})



> Entering new AgentExecutor chain...
반갑습니다, 유태영님! 어떤 메뉴의 레시피를 찾고 싶으신지 알려주시면 바로 도와드리겠습니다. 3번째 메뉴가 어떤 것인지 구체적으로 말씀해 주세요.

> Finished chain.


{'input': '나는 유태영이야',
 'chat_history': [HumanMessage(content='방금말한 메뉴중에 3번째 메뉴 레시피 찾아서 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='물론입니다. "3번째 메뉴"가 어떤 메뉴인지 구체적으로 알려주시면, 해당 메뉴의 레시피를 찾아드리겠습니다. 메뉴명을 알려주시겠어요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='나는 유태영이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반갑습니다, 유태영님! 어떤 메뉴의 레시피를 찾고 싶으신지 알려주시면 바로 도와드리겠습니다. 3번째 메뉴가 어떤 것인지 구체적으로 말씀해 주세요.', additional_kwargs={}, response_metadata={})],
 'output': '반갑습니다, 유태영님! 어떤 메뉴의 레시피를 찾고 싶으신지 알려주시면 바로 도와드리겠습니다. 3번째 메뉴가 어떤 것인지 구체적으로 말씀해 주세요.'}